# Road Following by Regression - Live Demo

In this notebook, we will try to make the JetBot to follow the desired road by using the trained model.

## Model Architecture
First we define a DNN model. This needs to be identical to what used for training.

In [1]:
import torch
import torch.nn as nn
device = torch.device('cuda')

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),    # dropout layer
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)  
        return x
    
model = Model().to(device)
print(model)

Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3136, out_features=256, bias

## Load The Best Model
Next we need to upload the `best_model_reg.pth` in the file browser. Then load the parameters on the model from the `best_model_reg.pth`.

In [2]:
model.load_state_dict(torch.load('best_model_reg.pth'))

<All keys matched successfully>

## Preprocessing Function

Now we create a function for preprocessing image data taken by the camera.

In [3]:
import cv2
import numpy as np
import torchvision

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

## Camera Instance
Now we create a camera instance.

In [4]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

size = 224
image = widgets.Image(format='jpeg', width=size, height=size)
camera = Camera.instance(width=size, height=size)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Robot Instance
Create the robot instance to drive the motors.

In [5]:
from jetbot import Robot
import time

robot = Robot()
robot.set_motors(-0.1, 0.1)
time.sleep(0.1)
robot.stop()

## Define move_robot function
Define a function to control the motors depending on the x and y coordinates of the target point.

In [6]:
def move_robot(x, y):
    # move robot
    speed_gain = 0.3
    b = 0.3               # kind of steering gain
    x = x/size-0.5
    y = 1-y/size
    a = np.sqrt(x**2 + y**2)
    angle = np.pi/2-np.arctan2(y, x)+1E-6
    c = a/np.sin(np.abs(angle))

    if angle >= 0:
        left_motor = speed_gain*(c+b)*np.abs(angle)
        right_motor = speed_gain*(c-b)*np.abs(angle)  
    else:
        left_motor = speed_gain*(c-b)*np.abs(np.abs(angle))
        right_motor = speed_gain*(c+b)*np.abs(np.abs(angle))

    robot.set_motors(left_motor, right_motor)
    
    return left_motor, right_motor

## Inference
Let's try to make an inference. This process takes for a while for the first time because it needs to load a model.

In [ ]:
import torch.nn.functional as F

with torch.no_grad():
    model.eval()
    img = camera.value
    img = preprocess(img)
    xy = size*model(img)[0].to("cpu").numpy()
    x = int(xy[0])
    y = int(xy[1])
    left_motor, right_motor = move_robot(x, y)
    time.sleep(0.1)
    robot.stop()
    print(f"left_motor={left_motor}, right_motor={right_motor}")

## Run JetBot
Run JetBot with a loop.

In [ ]:
t0 = time.time()

display(image)
steps = 100

with torch.no_grad():
    model.eval()

    for i in range(steps):
        try:
            img = camera.value
            img = preprocess(img)

            xy = size*model(img)[0].to("cpu").numpy()f
            x = int(xy[0])
            y = int(xy[1])

            left_motor, right_motor = move_robot(x, y)
            
        except:
            pass

        now = time.time()
        dt = now-t0
        t0 = now
        FPS = 1/dt

        print(f"\rStep:{i+1}/{steps}, left_motor={left_motor:.3f}, right_motor={right_motor:.3f}, FPS:{FPS:.1f}", end="")

robot.stop()

## Stop robot & camera

If you are done, stop the robot and the camera.

In [ ]:
robot.stop()
camera.stop()